In [1]:
import mysql.connector as sql
import pandas as pd
import datetime as dt
import numpy as np
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import matplotlib.pyplot as plt
from scipy.stats.mstats import mode
import json
import pickle
from itertools import chain, combinations
from lenspy import DynamicPlot
import plotly.graph_objects as go

In [2]:
db_connection = sql.connect(host='localhost', database='collection_visits', user='pandas', password='password')

visitsItems = pd.read_sql('SELECT * FROM VisitsItems', con=db_connection)
visitsItems['created'] = pd.to_datetime(visitsItems['created'], unit='s')
visitsItems['lastmodified'] = pd.to_datetime(visitsItems['lastmodified'], unit='s')
visitsItems['deleted'].replace(0, np.nan, inplace=True)
visitsItems['deleted'] = pd.to_datetime(visitsItems['deleted'], unit='s')
visitsItems['title'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
visitsItems['description'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
visitsItems['title'] = visitsItems['title'].astype('category')
visitsItems['description'] = visitsItems['description'].astype('category')

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option("max_colwidth", 500)

In [109]:
visitsItems_small = visitsItems[['user_id', 'refers_to_uid', 'created']]
visitsItems_small = visitsItems_small[visitsItems_small['user_id'] != 0]
visitsItems_small.set_index('created', inplace=True)
visitsItems_opening_hours = visitsItems_small.between_time('10:00', '18:00')
visitsItems_opening_hours.reset_index(inplace=True)

In [110]:
visitsItems_list = visitsItems_opening_hours.groupby('user_id')['refers_to_uid'].apply(pd.Series.tolist).tolist()

# file = json.dumps(visitsItems_list)
# text_file = open("listoflist.txt", "w")
# text_file.write(file)
# text_file.close()

In [111]:
visitsItems_list

[[18476565,
  18394375,
  18489807,
  18500009,
  18620817,
  18648865,
  18710413,
  18220487,
  68731001,
  68244063,
  68731009,
  68268059,
  35460835,
  18705931,
  68245549,
  35456853,
  18649321,
  51497635,
  51866827,
  68268235,
  68268237,
  68268239,
  68268241,
  68268243,
  68268245,
  68268247,
  68268251,
  68268253,
  68268255,
  68268233,
  68268227,
  68268229,
  68743599,
  18703483,
  35520989,
  18639109,
  35457285,
  35460703,
  35460713,
  51689393,
  35457319,
  35457321,
  35457317,
  35520541,
  18325257,
  35460389,
  35520549,
  68268461,
  35457109,
  18802401,
  18503039,
  18730767,
  18805891,
  68731001,
  68244063,
  68731009,
  68268059,
  35460835,
  18705931,
  68245549,
  35456853,
  18649321,
  51497635,
  51866827,
  68268235,
  68268237,
  68268239,
  68268241,
  68268243,
  68268245,
  68268247,
  68268251,
  68268253,
  68268255,
  68268233,
  68268227,
  68268229,
  68743599,
  18703483,
  35520989,
  18639109,
  35457285,
  35460703,
  35

In [4]:
  f = open("/Users/shirin/other_projects/cooper_hewitt/Pen_Data_Dictionary/relation_counts.json", "rb")
  data = pickle.load(f)
  f.close()
#this calculation exist in listtolist.py file

# def powerset(iterable):
#     "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
#     s = list(iterable)
#     return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
# list(powerset(visitsItems_list[2]))

In [5]:
relation_items = pd.DataFrame.from_dict(data,orient='index')
relation_items

,0
"(68268461, 18802401, 68268251)",2
"(35457321, 18325257, 68268241, 68268227)",2
"(35457109, 51866827)",2
"(35520549, 18802401, 68268243, 68268229)",2
"(35457321, 35460389, 35520549, 68268235)",2
...,...
"(51689393, 18325257, 35457109, 18802401)",29
"(35457317, 35520541, 35460389)",29
"(35457317, 18325257, 35460389, 18802401)",29
"(51689393, 35457319, 18325257, 18802401)",29


In [6]:
relation_items.sort_values(by=0, ascending=False, inplace=True)
relation_items.rename({0:'relation_count'}, axis=1, inplace=True)

In [7]:
relation_items.reset_index(inplace=True)

In [18]:
relation_items.iloc[800]

index             (51682607, 51682601, 51682595)
relation_count                                21
Name: 800, dtype: object

In [84]:
relation_items29 =relation_items[relation_items['relation_count'] > 14]
relation_items29.sample(10)

,index,relation_count
1967,"(69155389, 69155403)",17
77,"(35457321, 35457317, 18325257, 35460389)",29
1983,"(69155399, 69155395, 69155383)",17
2311,"(69155399, 69155403)",17
22,"(51689393, 35457321, 35520541, 35457109)",29
2609,"(69155385, 69155405, 69155403, 69155401)",17
2667,"(135918447, 135918431, 135918427, 135918421)",17
1049,"(18491031, 35460769, 35460771, 18349933)",20
1927,"(135918447, 135918443, 69155277)",17
188,"(35457321, 68268461)",29


In [ ]:
# relation_items.plot.bar(x='index', y='relation_count', rot=0)
relation_items.plot()